# CS246 - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 43.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=62b51a2c13571f5c0a33263bcc38fd7caa2508a53e4543dee4d38d5ff49275a0
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [42]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import regex as re
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sc.textFile("testo.txt") => RDD[String]

In [69]:
shakespeare = sc.textFile("pg100.txt")

In [70]:
def clean_line(str):
  lowercase = str.lower()
  lowercase = re.sub("[^0-9a-zA-Z]+", " ", lowercase)
  return lowercase

In [72]:
shakespeare_txt = shakespeare.map(clean_line)\
.flatMap(lambda st: st.split(" "))\
.filter(lambda x: x != '')\
.map(lambda word: (word[0], 1))

count_words = shakespeare_txt.reduceByKey(lambda a,b: (a+b))
count_words = count_words.sortBy(lambda x: -x[1])

count_words.take(36)

[('t', 127781),
 ('a', 86000),
 ('s', 75226),
 ('i', 62420),
 ('h', 61029),
 ('w', 60097),
 ('m', 56252),
 ('b', 46001),
 ('o', 43712),
 ('d', 39173),
 ('f', 37186),
 ('c', 34983),
 ('l', 32389),
 ('p', 28059),
 ('n', 27313),
 ('y', 25926),
 ('g', 21167),
 ('e', 20409),
 ('r', 15234),
 ('k', 9535),
 ('u', 9230),
 ('v', 5802),
 ('j', 3372),
 ('q', 2388),
 ('1', 932),
 ('2', 330),
 ('3', 84),
 ('z', 79),
 ('4', 67),
 ('5', 43),
 ('9', 34),
 ('6', 28),
 ('7', 24),
 ('8', 23),
 ('x', 14),
 ('0', 10)]

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!